In [10]:
import processGFF3 as pg
import processFa as pf
from Bio.SeqFeature import SeqFeature, FeatureLocation, ExactPosition, SimpleLocation
from Bio.SeqRecord import SeqRecord
import utils as ut
import subprocess
import os
import requests

## Set up species and genomeBuild

In [11]:
#GenomeBuild = "GRCh38" # Human
GenomeBuild = "GRCm39" # Mouse
#GenomeBuild = "GRCz11" # Zebrafish

#species = "Homo_sapiens" # Human
species = "Mus_musculus" # Mouse
#species = "Danio_rerio" # Zebrafish

gff3_path = f"{species}.{GenomeBuild}.107.gff3.gz"
fasta_path = f"{species}.{GenomeBuild}.dna_sm.primary_assembly.fa.gz"
out_EI_fa_path = f"Exon_intron_junc_seqs.{GenomeBuild}.fa"
out_IE_fa_path = f"Intron_exon_junc_seqs.{GenomeBuild}.fa"
out_EI_weblogo_path = f"Exon_intron_junc_seqs.{GenomeBuild}.weblogo.eps"
out_IE_weblogo_path = f"Intron_exon_junc_seqs.{GenomeBuild}.weblogo.eps"

## Load and parse GFF3

In [12]:
#check if files exist, download if not
if os.path.isfile(gff3_path) == False:
    url = f"https://ftp.ensembl.org/pub/release-107/gff3/{species.lower()}/{gff3_path}"
    print(f"Downloading {url}")
    response = requests.get(url, stream=True)
    with open(gff3_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=204800):
            if chunk:
                f.write(chunk)

ENST_info = pg.parse_gff3(gff3_path)
print("-done...")

-processing ENST IDs
-processing exons
-processing cds
-done...


## Load and parse genome fasta

In [13]:
#check if files exist, download if not
if os.path.isfile(fasta_path) == False:
    url = f"https://ftp.ensembl.org/pub/release-107/fasta/{species.lower()}/dna/{fasta_path}"
    print(f"Downloading {url}")
    response = requests.get(url, stream=True)
    with open(fasta_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=204800):
            if chunk:
                f.write(chunk)

print("-loading genome sequence...")
genome_seqs = pf.parse_fasta(fasta_path)
print("-done...")

-loading genome sequence...
-done...


## Extract exon junctions

In [14]:
counter = 0
with open(out_EI_fa_path, "w") as ei, open(out_IE_fa_path, "w") as ie:

    for ENST in ENST_info.keys():
        features = ENST_info[ENST].features
        exons = [i for i in features if i.type=="exon"]
        sorted_exons = ut.sort_exons(exons)

        EI_junc_coord = ut.extract_exon_intron_junc_coord(sorted_exons)
        IE_junc_coord = ut.extract_intron_exon_junc_coord(sorted_exons)
        
        EI_junc_seqs = ut.extract_junc_seq(EI_junc_coord, genome_seqs, left_padding = 20, right_padding = 20)   
        IE_junc_seqs = ut.extract_junc_seq(IE_junc_coord, genome_seqs, left_padding = 20, right_padding = 20)

        for index, seq in enumerate(EI_junc_seqs):
            ei.write(f">{ENST}_{index}\n{str(seq)}\n")

        for index, seq in enumerate(IE_junc_seqs):
            ie.write(f">{ENST}_{index}\n{str(seq)}\n")

        counter += 1
        if counter % 5000 == 0:
            print(counter, " ENSTs processed")

5000  ENSTs processed
10000  ENSTs processed
15000  ENSTs processed
20000  ENSTs processed
25000  ENSTs processed
30000  ENSTs processed
35000  ENSTs processed
40000  ENSTs processed
45000  ENSTs processed
50000  ENSTs processed
55000  ENSTs processed
60000  ENSTs processed
65000  ENSTs processed
70000  ENSTs processed
75000  ENSTs processed
80000  ENSTs processed
85000  ENSTs processed
90000  ENSTs processed
95000  ENSTs processed
100000  ENSTs processed
105000  ENSTs processed
110000  ENSTs processed
115000  ENSTs processed
120000  ENSTs processed
125000  ENSTs processed
130000  ENSTs processed
135000  ENSTs processed
140000  ENSTs processed
145000  ENSTs processed


In [15]:
subprocess.run(["weblogo", "-f", out_EI_fa_path, "-D", "fasta", "-F", "eps","-o", out_EI_weblogo_path])
subprocess.run(["weblogo", "-f", out_IE_fa_path, "-D", "fasta", "-F", "eps","-o", out_IE_weblogo_path])

CompletedProcess(args=['weblogo', '-f', 'Intron_exon_junc_seqs.GRCm39.fa', '-D', 'fasta', '-F', 'eps', '-o', 'Intron_exon_junc_seqs.GRCm39.weblogo.eps'], returncode=0)